<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Pytorch_WithDeeplizard/blob/master/38_PyTorch_Sequential_Models_Neural_Networks_Made_Easy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Sequential Models - Neural Networks Made Easy
在本节中，我们将学习如何使用PyTorch的Sequential类来构建神经网络。

## 1.PyTorch Sequential 模块

序列类使我们能够即时构建PyTorch神经网络，而无需构建显式类。 这使快速构建网络变得容易得多，并允许我们跳过实现theforward（）方法的步骤。 当我们使用顺序的方式构建PyTorch网络时，我们通过顺序定义网络的体系结构隐式地构造了forward（）方法。

顺序模块是扩展nn.Module基类并允许我们将模块组合在一起的容器或包装类。 我们可以在任何其他nn.Module中组成任何nn.Module。

这意味着我们可以构成网络的各层，并且由于网络也是nn.Module实例，因此我们也可以彼此构成网络。 另外，由于Sequential类本身也是nn.Module，因此我们甚至可以相互组合Sequential模块。

在这一点上，我们可能想知道其他所需的功能和操作，例如池化操作或激活功能。 答案是，nn.functional API中的所有功能和操作都被包装到nn.Module类中。 这使我们可以将激活功能之类的东西传递给顺序包装器，从而以顺序方式完全构建我们的网络。



## 2.Building PyTorch Sequential Networks
有三种创建顺序模型的方法。 让我们看看它们的作用。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import math

from collections import OrderedDict

torch.set_printoptions(linewidth=150)

然后，我们需要创建一个数据集，我们可以将其用于将样本传递到将要构建的网络中。

In [ ]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


现在，我们将从FashionMNIST数据集实例中获取示例图像。

In [ ]:
image,label = train_set[0]
image.shape

torch.Size([1, 28, 28])

现在，我们将获取一些用于构建我们的网络的值

In [ ]:
in_features = image.numel()
in_features

784

In [ ]:
out_features = math.floor(in_features / 2)
out_features

392

In [ ]:
out_classes = len(train_set.classes)
out_classes

10

### Sequential Model Initialization: Way 1
--- 
创建顺序模型的第一种方法是将nn.Module实例直接传递给Sequential类构造函数。

In [ ]:
network1 = nn.Sequential(
    nn.Flatten(start_dim=1),
    nn.Linear(in_features,out_features),
    nn.Linear(out_features,out_classes)
)

### Sequential Model Initialization: Way 2
---
创建顺序模型的第二种方法是创建一个包含nn.Module实例的OrderedDict。 然后，将字典传递给Sequential类构造函数。

In [ ]:
layers = OrderedDict(
  [
   ('flat',nn.Flatten(start_dim=1)),
   ('hidden',nn.Linear(in_features,out_features)),
   ('output',nn.Linear(out_features,out_classes))
  ]
)

print(layers)

OrderedDict([('flat', Flatten(start_dim=1, end_dim=-1)), ('hidden', Linear(in_features=784, out_features=392, bias=True)), ('output', Linear(in_features=392, out_features=10, bias=True))])


In [ ]:
network2 = nn.Sequential(layers)

这种初始化方式使我们可以显式命名nn.Module实例。

### Sequential Model Initialization: Way 3
---
创建序列模型的第三种方法是使用空构造函数创建序列实例。然后，我们可以使用add_module（）方法来添加nn.Module在初始化网络后将实例添加到网络。

In [ ]:
network3 = nn.Sequential()
network3.add_module('flat',nn.Flatten(start_dim=1))
network3.add_module('hidden',nn.Linear(in_features,out_features))
network3.add_module("output",nn.Linear(out_features,out_classes))

这种初始化方式还允许我们显式命名nn.Module实例。

## 3.Class Definition Vs Sequential
到目前为止，在本课程中，我们一直在使用一个类定义来定义CNN。网络的定义如下：

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 12, 5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):

        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)

        return t

我们得到这样的网络实例：

In [ ]:
network = Network()

现在，让我们看看如何使用Sequential类创建相同的网络。 它是这样的：

In [ ]:
sequential = nn.Sequential(
      nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)
    , nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    , nn.ReLU()
    , nn.MaxPool2d(kernel_size=2, stride=2)
    , nn.Flatten(start_dim=1)  
    , nn.Linear(in_features=12*4*4, out_features=120)
    , nn.ReLU()
    , nn.Linear(in_features=120, out_features=60)
    , nn.ReLU()
    , nn.Linear(in_features=60, out_features=10)
)

我们说这些网络是一样的。但我们的意思是什么呢？在这种情况下，我们的意思是网络具有相同的架构。从编程的角度来看，这两个网络是不同类型的。

注意，如果我们修正用于在 PyTorch 中生成随机数的种子，就可以得到这两个网络相同的输出预测。这是因为两个网络的权重都是随机产生的。为了确保权重相同，我们在创建每个网络之前使用下面的 PyTorch 方法。

In [ ]:
torch.manual_seed(50)

需要注意的是，该方法必须调用两次，每次网络初始化之前调用一次。